In [28]:
# Here I will be building out the architecture of the first classification LSTM
# At each time step, this LSTM will take in a vector representing the extracted audio and visual features from Liris-ACCEDE
# Its goal is to output whether or not the movie induces fear at each time step

# First, import necessary libraries
import tensorflow as tf
import numpy as np

In [29]:
# setting up the keras stuff
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM

In [30]:
# create array of shape (210, 4096, 1) that will hold each second's worth of VGG16 data
X_input = np.zeros([210, 4096, 1]) # MAGIC NUMBERS

# iterate through the first axis length of this array and insert each second's worth of fc6 feature data
# start counting seconds at 00001

# uploading the X_values: fc6 feature data as input and figuring out its shape
input_data = np.loadtxt('visual_features_part01/07/fc6/MEDIAEVAL18_07-00001_fc6.txt', delimiter=',')
print(input_data.shape)


(4096,)


In [51]:
# uploading the Y_values
# the target data (y_values) will be a one-hot vector representing which seconds of movie induce fear

# convert into function that takes following input args: movie_length, fear_annot_path
# returns y_data_input

# first access start and stop times for fear-inducing sequences
y_data = np.loadtxt('fear_annotations_part01/MEDIAEVAL18_07_Fear.txt', skiprows=1)

# now treat these as pairs of indices --> we want all the indices between each pair of numbers
# create array of zeros --> the size will be the number of seconds in movie, in this case 210
movie_length = 212 #MAGIC NUMBER ALERT! --> length of movie
y_data_input = np.zeros((movie_length)) 

# for each element in first dimension of the y_data array
for i in range(y_data.shape[0]):
    # access the start time number and end time number
    start = int(y_data[i][0])
    end = int(y_data[i][1])
    # set the elements between these indices in the zeros array to one
    y_data_input[start] = 1 #maybe superfluous
    y_data_input[end] = 1
    y_data_input[start:end] = 1
print(y_data_input)

[[  64.  101.]
 [ 105.  109.]
 [ 129.  145.]]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.  1.  1.  1.
  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [52]:
# setting up some key values
timesteps = 212   # the number of seconds in movie 07 --> i will figure out how to programmatically get this value
data_dim = 4096    # the number of output values from VGG16 layer fc6 --> switch to programmatic later
# could data_dim be the number of features that have been extracted (for now visual features only) --> maybe too much

# I have yet to figure this out
X_train = input_data
Y_train = y_data_input
batch_size = 30 # very much arbitrary
num_epochs = 20 # very much arbitrary

In [53]:
# constructing a many-to-one LSTM model in keras --> inspiration: https://stackoverflow.com/questions/43034960/many-to-one-and-many-to-many-lstm-examples-in-keras
# i will start by training a model on only the VGG16 fc6 layer output (that's just one feature)
model = Sequential()
model.add(LSTM(1, input_shape=(timesteps, data_dim), return_sequences=True))
# going to add a softmax activation to this
model.add(Activation('softmax'))

In [54]:
# compiling LSTM model
# note that Ng used an Adam optimizer and categorical cross-entropy loss
# but this is a binary classification problem so I think the parameters below should suffice
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [55]:
# running the LSTM model
model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs)
print("finished training!")

ValueError: Error when checking input: expected lstm_2_input to have 3 dimensions, but got array with shape (4096, 1)